In [ ]:
# Runs in CV3 pyenv

import numpy as np
import cv2
import os

In [ ]:
video_directory = 'raw-video'

In [ ]:
video_file_name = 'Wildlife.mp4'

In [ ]:
num_frames_per_cycle = 5

In [ ]:
# Generator returns (success_flag, frame_number, left_frame, middle_frame, right_frame)
def get_next_N_frames(vid_cap, num_frames = 3):
    counter = 0
    ret_frame_list = list([(None, None, None) for i in range(num_frames)])
    
    # Read the first num_frames-1 frames without yielding
    for initial_frame_index in range(num_frames-1):
        if vid_cap.isOpened():
            
            skip_this_frame = True
            while(skip_this_frame):
                ret, frame = vid_cap.read()                
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                frame = cv2.resize(frame, (256, 256))
                print(cv2.countNonZero(frame))

                if cv2.countNonZero(frame) == 0:
                    skip_this_frame = True
                else:
                    skip_this_frame = False
                    
            counter += 1
    
            ret_frame_list.append((ret, frame, counter))
            ret_frame_list.pop(0)
        else:
            yield False, None, [None for ret_frame in ret_frame_list]
    
    while(vid_cap.isOpened()):
        # Append the next frame to the list, and drop the first frame
        ret, frame = vid_cap.read()
        counter += 1

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame = cv2.resize(frame, (256, 256))

        ret_frame_list.append((ret, frame, counter))
        ret_frame_list.pop(0)
        
        if all([ret_frame[0] for ret_frame in ret_frame_list]):
            yield True, counter, [(ret_frame[1], ret_frame[2]) for ret_frame in ret_frame_list]
        else:
            yield False, counter, [None for ret_frame in ret_frame_list]

    yield False, None, [None for ret_frame in ret_frame_list]

In [ ]:
video_capture = cv2.VideoCapture(os.path.join(video_directory, video_file_name))
frame_generator = get_next_N_frames(video_capture, num_frames_per_cycle)

In [ ]:
# Step through the video frame by frame (re-execute this cell for each new frame)
frame_success, frame_number, frame_list = next(frame_generator)

if frame_success:
    offset_x = 0
    offset_y = 0
    
    for frame_number in frame_list:
        cv2.namedWindow('frame: ' + str(frame_number[1]))
        cv2.imshow('frame: ' + str(frame_number[1]), frame_number[0])
        cv2.moveWindow('frame: ' + str(frame_number[1]), offset_x, offset_y)
        offset_x += 200
        offset_y += 50
    k = cv2.waitKey(500)
    cv2.destroyAllWindows()
else:
    if frame_number is None:
        print('Failed to extract any frames')

In [ ]:

video_capture.release()
